In [2]:
# Cell 1: Setup and Imports - Discover ALL Codes
import sys
import time
import numpy as np
import warnings
from IPython.display import clear_output

clear_output(wait=True)
warnings.filterwarnings('ignore')

# Force reload of modules
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

import stim
from qectostim.codes import discover_all_codes
from qectostim.codes.abstract_code import StabilizerCode
from qectostim.codes.abstract_css import CSSCode
from qectostim.experiments.memory import (
    CSSMemoryExperiment, 
    StabilizerMemoryExperiment,
    ColorCodeMemoryExperiment,  # NEW: For Chromobius-compatible color codes
)
from qectostim.noise.models import CircuitDepolarizingNoise

# Discover ALL codes including subsystem, floquet, and QLDPC
discovered = discover_all_codes(
    max_qubits=100, 
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True
)

# Categorize codes by type - use dict to prevent duplicates
css_codes = {}
non_css_codes = {}
subsystem_codes = {}
floquet_codes = {}
qldpc_codes = {}
color_codes = {}  # NEW: Color codes get special treatment for Chromobius

for name, code in discovered.items():
    # Check if it's a color code (has Chromobius-compatible metadata)
    meta = getattr(code, 'metadata', {}) if hasattr(code, 'metadata') else {}
    is_color_code = meta.get('is_chromobius_compatible', False)
    
    if 'Floquet' in name or 'Honeycomb' in name or 'ISG' in name:
        floquet_codes[name] = code
    elif 'Subsystem' in name or 'Gauge' in name or 'Bacon' in name:
        subsystem_codes[name] = code
    elif any(x in name for x in ['HGP', 'BB', 'GB', 'Hypergraph', 'Bicycle', 'Lifted', 'Fiber']):
        qldpc_codes[name] = code
    elif is_color_code:
        color_codes[name] = code  # Color codes (Chromobius-compatible)
    elif hasattr(code, 'hx') and hasattr(code, 'hz'):
        css_codes[name] = code
    elif hasattr(code, 'stabilizer_matrix'):
        non_css_codes[name] = code

# Combine ALL codes with type markers
all_codes = {}
for name, code in css_codes.items():
    all_codes[name] = ('CSS', code)
for name, code in non_css_codes.items():
    all_codes[name] = ('Non-CSS', code)
for name, code in subsystem_codes.items():
    all_codes[name] = ('Subsystem', code)
for name, code in floquet_codes.items():
    all_codes[name] = ('Floquet', code)
for name, code in qldpc_codes.items():
    all_codes[name] = ('QLDPC', code)
for name, code in color_codes.items():
    all_codes[name] = ('Color', code)  # NEW type

# Build output as single string to prevent duplication
output = []
output.append("="*70)
output.append("DECODER SMOKE TEST - ALL CODE TYPES")
output.append("="*70)
output.append(f"\nTotal discovered: {len(discovered)} codes")
output.append(f"  CSS:       {len(css_codes)}")
output.append(f"  Color:     {len(color_codes)} (Chromobius-compatible)")
output.append(f"  Non-CSS:   {len(non_css_codes)}")
output.append(f"  Subsystem: {len(subsystem_codes)}")
output.append(f"  Floquet:   {len(floquet_codes)}")
output.append(f"  QLDPC:     {len(qldpc_codes)}")

for title, codes_dict in [("CSS Codes", css_codes), ("Color Codes (Chromobius)", color_codes),
                           ("Non-CSS Codes", non_css_codes), ("Subsystem Codes", subsystem_codes), 
                           ("Floquet Codes", floquet_codes), ("QLDPC Codes", qldpc_codes)]:
    output.append(f"\n{title}:")
    output.append(f"{'Code Name':<40} {'n':>4} {'k':>3} {'d':>3}")
    output.append("-"*55)
    for name, code in codes_dict.items():
        d = code.metadata.get('distance', '?')
        output.append(f"  {name:<38} {code.n:>4} {code.k:>3} {d:>3}")
    output.append(f"\n{len(codes_dict)} codes")

print("\n".join(output))

DECODER SMOKE TEST - ALL CODE TYPES

Total discovered: 65 codes
  CSS:       48
  Color:     4 (Chromobius-compatible)
  Non-CSS:   5
  Subsystem: 4
  Floquet:   1
  QLDPC:     3

CSS Codes:
Code Name                                   n   k   d
-------------------------------------------------------
  FourQubit422_[[4,2,2]]                    4   2   2
  C6                                        6   2   2
  Steane_713                                7   1   3
  Shor_91                                   9   1   3
  ReedMuller_15_1_3                        15   1   3
  Hamming_CSS_7                             7   1   3
  Code_832                                  8   3   2
  Repetition_3                              3   1   3
  Repetition_5                              5   1   5
  Repetition_7                              7   1   7
  RotatedSurface_[[9,1,3]]                  9   1   3
  RotatedSurface_[[25,1,5]]                25   1   5
  ToricCode_3x3                            18   2  

In [13]:
# Debug cell: Check QLDPC factory loading
from qectostim.codes import qldpc as qldpc_module

print("Checking QLDPC factories in qldpc module:")
qldpc_factories = ['HGPHamming7', 'BBGrossCode', 'HDX_4', 'HDX_6', 'QuantumTanner_4', 'DLV_8',
                   'ExpanderLP_10_3', 'DHLV_5_1', 'BalancedProductRep5']

for name in qldpc_factories:
    factory = getattr(qldpc_module, name, None)
    if factory is None:
        print(f"  ❌ {name}: NOT FOUND in qldpc module")
    else:
        try:
            code = factory()
            print(f"  ✅ {name}: Success - {type(code).__name__} with n={getattr(code, 'n', '?')}")
        except Exception as e:
            print(f"  ⚠️ {name}: Found but failed - {e}")

# Also check if discovery.py's qldpc_names pattern would match these
print("\nChecking if QLDPC patterns would match:")
qldpc_names = ['Hypergraph', 'Bicycle', 'Lifted', 'Fiber', 'HGP', 'BB', 'GB']
for name in ['HDXCode', 'ExpanderLPCode', 'DHLVCode', 'QuantumTannerCode', 'DinurLinVidickCode']:
    matches = any(pat in name for pat in qldpc_names)
    print(f"  {name}: {'✅' if matches else '❌'} matches QLDPC pattern")

ImportError: cannot import name 'HGPHamming7' from 'qectostim.codes.qldpc.hypergraph_product' (/Users/scottjones_admin/Library/Mobile Documents/com~apple~CloudDocs/Mac files/Repos/QECToStim/src/qectostim/codes/qldpc/hypergraph_product.py)

In [3]:
# Cell 2: Load Available Decoders

decoder_classes = {}

# PyMatching
try:
    from qectostim.decoders.pymatching_decoder import PyMatchingDecoder
    decoder_classes['PyMatching'] = PyMatchingDecoder
except: pass

# Fusion Blossom
try:
    from qectostim.decoders.fusion_blossom_decoder import FusionBlossomDecoder
    decoder_classes['FusionBlossom'] = FusionBlossomDecoder
except: pass

# Belief Matching
try:
    from qectostim.decoders.belief_matching import BeliefMatchingDecoder
    decoder_classes['BeliefMatching'] = BeliefMatchingDecoder
except: pass

# BP-OSD
try:
    from qectostim.decoders.bp_osd import BPOSDDecoder
    decoder_classes['BPOSD'] = BPOSDDecoder
except: pass

# Tesseract
try:
    from qectostim.decoders.tesseract_decoder import TesseractDecoder
    decoder_classes['Tesseract'] = TesseractDecoder
except: pass

# Union Find (fallback to PyMatching)
try:
    from qectostim.decoders.union_find_decoder import UnionFindDecoder
    decoder_classes['UnionFind'] = UnionFindDecoder
except: pass

# MLE Decoder (exact, for small codes)
try:
    from qectostim.decoders.mle_decoder import MLEDecoder
    decoder_classes['MLE'] = MLEDecoder
except: pass

# Hypergraph Decoder (PyMatching + boundary L0 correction)
try:
    from qectostim.decoders.mle_decoder import HypergraphDecoder
    decoder_classes['Hypergraph'] = HypergraphDecoder
except: pass

# Chromobius Decoder (for hyperedge DEMs) 
try:
    from qectostim.decoders.chromobius_decoder import ChromobiusDecoder
    decoder_classes['Chromobius'] = ChromobiusDecoder
except: pass

print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

Loaded 9 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius']


In [4]:
# Cell 3: Test Helper Function - Uses ColorCodeMemoryExperiment for Color Codes

# Import ChromobiusIncompatibleError for proper handling
try:
    from qectostim.decoders.chromobius_decoder import ChromobiusIncompatibleError
except ImportError:
    ChromobiusIncompatibleError = Exception  # Fallback

def test_decoder_on_code(code, code_type, decoder_name, decoder_class, p=0.01, shots=1000, rounds=3):
    """Test a decoder on ANY code type and return results dict.
    
    For color codes (code_type='Color'), uses ColorCodeMemoryExperiment to generate
    Chromobius-compatible DEMs with 4D detector coordinates.
    
    Chromobius will SKIP on non-color codes (expected behavior).
    """
    result = {
        'status': 'UNKNOWN',
        'ler': None,
        'ler_no_decode': None,
        'time_ms': None,
        'warnings': [],
        'error': None
    }
    
    try:
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        
        # Use appropriate experiment class based on code type
        is_css_code = hasattr(code, 'hx') and hasattr(code, 'hz')
        
        if code_type == 'Color':
            # Color codes use ColorCodeMemoryExperiment for Chromobius-compatible DEMs
            exp = ColorCodeMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        elif is_css_code:
            exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        else:
            # Non-CSS, Subsystem, Floquet, QLDPC all use StabilizerMemoryExperiment
            exp = StabilizerMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        
        circuit = noise.apply(exp.to_stim())
        
        # Try to build DEM - with fallback for decomposition failures
        try:
            dem = circuit.detector_error_model(decompose_errors=True)
        except Exception as e1:
            try:
                dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
            except Exception as e2:
                # Check for non-deterministic observable (mixed logical operators)
                err_msg = str(e2)
                if 'non-deterministic' in err_msg.lower():
                    result['status'] = 'WARN'
                    result['error'] = "Mixed logical ops"
                    result['warnings'].append('Mixed logical ops')
                else:
                    result['status'] = 'FAIL'
                    result['error'] = f"DEM: {err_msg[:30]}"
                return result
        
        # Sample
        sampler = dem.compile_sampler()
        raw = sampler.sample(shots, bit_packed=False)
        
        if isinstance(raw, tuple):
            det_samples = np.asarray(raw[0], dtype=np.uint8)
            obs_samples = np.asarray(raw[1], dtype=np.uint8)
        else:
            arr = np.asarray(raw, dtype=np.uint8)
            det_samples = arr[:, :dem.num_detectors]
            obs_samples = arr[:, dem.num_detectors:]
        
        if obs_samples.shape[1] > 0:
            result['ler_no_decode'] = float(obs_samples[:, 0].mean())
        
        # Create decoder - wrap in try/except to catch initialization failures
        try:
            if decoder_name in ['PyMatching', 'FusionBlossom']:
                decoder = decoder_class(dem)
            else:
                decoder = decoder_class(dem=dem)
        except ChromobiusIncompatibleError as e:
            # Chromobius requires color-code-like DEMs - this is expected for non-color codes
            result['status'] = 'SKIP'
            result['error'] = f"Not a color code"
            result['warnings'].append('Chromobius: requires color-code DEM')
            return result
        except Exception as e:
            err_msg = str(e)
            # Check if it's a Chromobius-related error
            if 'chromobius' in err_msg.lower() or 'color' in err_msg.lower():
                result['status'] = 'SKIP'
                result['error'] = f"Not a color code"
                result['warnings'].append('Chromobius: requires color-code DEM')
            else:
                result['status'] = 'FAIL'
                result['error'] = f"Decoder init: {err_msg[:25]}"
            return result
        
        # Decode - wrap in try/except to catch decode failures (e.g., FusionBlossom panics)
        try:
            start = time.time()
            corrections = decoder.decode_batch(det_samples)
            result['time_ms'] = (time.time() - start) * 1000
        except BaseException as e:
            # Use BaseException to catch Rust PanicException from pyo3
            err_msg = str(e)
            err_type = type(e).__name__
            if 'panic' in err_type.lower() or 'matching' in err_msg.lower() or 'unmatched' in err_msg.lower():
                result['status'] = 'WARN'
                result['error'] = f"Decoder issue: {err_type}"
                result['warnings'].append('Decoder incompatibility')
            else:
                result['status'] = 'FAIL'
                result['error'] = f"Decode: {err_msg[:25]}"
            return result
        
        corrections = np.asarray(corrections, dtype=np.uint8)
        if corrections.ndim == 1:
            corrections = corrections.reshape(-1, max(1, dem.num_observables))
        
        if obs_samples.shape[1] > 0:
            logical_errors = (corrections[:, 0] ^ obs_samples[:, 0]).astype(np.uint8)
            result['ler'] = float(logical_errors.mean())
        
        result['status'] = 'OK'
        
        # Warnings
        ler, ler_nd = result['ler'], result['ler_no_decode']
        if ler is not None:
            if ler < 1e-6 and p > 0.001:
                result['warnings'].append('LER≈0')
            if ler_nd is not None and ler >= ler_nd and code.metadata.get('distance', 0) >= 3:
                result['warnings'].append('No improvement')
        
    except BaseException as e:
        # Use BaseException to catch ALL exceptions including Rust panics
        err_type = type(e).__name__
        if 'panic' in err_type.lower():
            result['status'] = 'WARN'
            result['error'] = f"Rust panic: {err_type}"
            result['warnings'].append('Rust decoder panic')
        else:
            result['status'] = 'FAIL'
            result['error'] = str(repr(e))[:30]
    
    return result

print("✓ Test helper defined")
print("  - Color codes use ColorCodeMemoryExperiment (Chromobius-compatible DEMs)")
print("  - Other codes use CSSMemoryExperiment or StabilizerMemoryExperiment")
print("  - Chromobius will SKIP on non-color codes (expected)")

✓ Test helper defined
  - Color codes use ColorCodeMemoryExperiment (Chromobius-compatible DEMs)
  - Other codes use CSSMemoryExperiment or StabilizerMemoryExperiment
  - Chromobius will SKIP on non-color codes (expected)


In [5]:
# Cell 4: DECODER × CODE COMPATIBILITY MATRIX - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

p = 0.01
shots = 1000

# Store results - reinitialize to be safe
full_results = {}

# Build output lines to print at end (prevents output duplication)
output_lines = []
output_lines.append("="*130)
output_lines.append("DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES")
output_lines.append("="*130)
output_lines.append(f"\nTesting {len(all_codes)} decoder/code combinations at p={p}, {shots} shots...")
output_lines.append("Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC")
output_lines.append("Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:^12}"
output_lines.append(header)
output_lines.append("-" * len(header))

# Process each code ONCE
for code_name, (code_type, code) in all_codes.items():
    row = f"{code_name:<35} {code_type:<10}"
    full_results[code_name] = {'type': code_type}
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=code,
            code_type=code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=shots
        )
        full_results[code_name][dec_name] = result
        
        if result['status'] == 'OK':
            ler = result['ler']
            if result['warnings']:
                cell = f"⚠️{ler:.4f}" if ler is not None else "⚠️N/A"
            else:
                cell = f"✓ {ler:.4f}" if ler is not None else "✓ N/A"    
        elif result['status'] == 'WARN':
            cell = f"⚠️WARN"
        elif result['status'] == 'SKIP':
            cell = f"- SKIP"
        else:
            cell = f"✗ FAIL"
        
        row += f" | {cell:^12}"
    
    output_lines.append(row)

output_lines.append("-" * len(header))
output_lines.append(f"\nLegend: ✓=pass, ⚠️=warning, -=skip (expected), ✗=fail")
output_lines.append(f"Total codes tested: {len(full_results)}")

# Single print statement
print("\n".join(output_lines))


thread '<unnamed>' (20031108) panicked at src/primal_module_serial.rs:1253:17:
cannot compute final matching with unmatched outer node 3
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES

Testing 65 decoder/code combinations at p=0.01, 1000 shots...
Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC
Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)

Code                                Type       |  PyMatching  |  FusionBlos  |  BeliefMatc  |    BPOSD     |  Tesseract   |  UnionFind   |     MLE      |  Hypergraph  |  Chromobius 
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
FourQubit422_[[4,2,2]]              CSS        |   ✓ 0.0460   |   ✓ 0.0590   |   ✓ 0.0450   |   ✓ 0.0480   |   ✓ 0.0360   |   ✓ 0.0600   |   ✓ 0.0470   |   ✓ 0.0450   |    - SKIP   
C6                                  CSS        |   ✓ 0.0690   |   ✓ 0.1000   |   ✓ 0.0740   |   ✓ 0.0980   |   ✓ 0.0730   |   ✓ 0.0790   |   ✓ 0.0860   |   ✓ 0.0890   |    - SKIP   
Steane_713                 

In [8]:
# Cell 5: LER COMPARISON TABLE - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

print("="*140)
print("LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES")
print("="*140)
print(f"\nTotal codes in all_codes: {len(all_codes)}")
print(f"Total codes in full_results: {len(full_results)}")
print("\nLower is better. Best decoder for each code highlighted.")
print("SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} | {'Type':<10} | {'d':>2} | {'No-decode':>10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
header += " | Best"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    
    # Get code object for distance
    code = all_codes.get(code_name, (None, None))[1]
    d = code.metadata.get('distance', '?') if code else '?'
    
    # Get no-decode LER
    ler_no_decode = None
    for key, res in code_results.items():
        if isinstance(res, dict) and res.get('ler_no_decode') is not None:
            ler_no_decode = res['ler_no_decode']
            break
    
    nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else 'N/A'
    row = f"{code_name:<35} | {code_type:<10} | {str(d):>2} | {nd_str:>10}"
    
    # Find best decoder
    best_ler = float('inf')
    best_decoder = None
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        status = res.get('status') if isinstance(res, dict) else None
        ler = res.get('ler') if isinstance(res, dict) else None
        
        if status == 'SKIP':
            ler_str = 'SKIP'
        elif ler is not None:
            ler_str = f"{ler:.4f}"
            if ler < best_ler:
                best_ler = ler
                best_decoder = dec_name
        else:
            ler_str = 'FAIL'
        row += f" | {ler_str:>10}"
    
    best_str = best_decoder[:10] if best_decoder else 'N/A'
    row += f" | {best_str}"
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES

Total codes in all_codes: 65
Total codes in full_results: 65

Lower is better. Best decoder for each code highlighted.
SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)

Code                                | Type       |  d |  No-decode | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius | Best
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BaconShor_3x3                       | Subsystem  |  3 |     0.0770 |     0.0220 |     0.0150 |     0.0170 |     0.0180 |     0.0170 |     0.0220 |     0.0200 |     0.0910 |       SKIP | FusionBlos
BalancedProduct_5x5_G1              | CSS        |  4 |     0.0240 |     0.0050 |     0.0060 |     0.0030 |     0.0040 |     0.0070 |     0.0030 |       FAIL |     0.0570 |       SK

In [6]:
# Cell 6: DECODER SPEED COMPARISON
from IPython.display import clear_output
clear_output(wait=True)

print("="*110)
print("DECODER SPEED COMPARISON (time in ms for 1000 shots)")
print("="*110)
print(f"\nTotal codes in full_results: {len(full_results)}\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<30} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    row = f"{code_name:<30} {code_type:<10}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        time_ms = res.get('time_ms') if isinstance(res, dict) else None
        time_str = f"{time_ms:.1f}" if time_ms else 'FAIL'
        row += f" | {time_str:>10}"
    
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

DECODER SPEED COMPARISON (time in ms for 1000 shots)

Total codes in full_results: 48

Code                           Type       | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius
--------------------------------------------------------------------------------------------------------------------------------------------------------------
BaconShor_3x3                  Subsystem  |        0.2 |        3.7 |       11.4 |       12.6 |       10.7 |        0.2 |        0.5 |        1.0 |       FAIL
BareAncilla_713                Non-CSS    |        0.1 |        3.1 |        6.4 |        6.0 |       11.1 |        0.1 |        0.4 |        0.9 |       FAIL
C6                             CSS        |        0.1 |        3.0 |        5.3 |        5.1 |       10.3 |        0.1 |        0.5 |        0.8 |       FAIL
Code_832                       CSS        |        0.3 |        5.5 |       19.5 |       50.3 |       15.0 |        0.

In [7]:
# Cell 7: NOISE LEVEL SCALING TEST

print("="*110)
print("NOISE LEVEL SCALING TEST")
print("="*110)

# Pick a representative CSS code (RotatedSurface d=3) 
test_code_name = None
test_code_type = None
for name, (ctype, code) in all_codes.items():
    if 'RotatedSurface' in name and code.metadata.get('distance') == 3:
        test_code_name = name
        test_code_type = ctype
        break

if test_code_name is None:
    test_code_name = list(all_codes.keys())[0]
    test_code_type = all_codes[test_code_name][0]

test_code = all_codes[test_code_name][1]
print(f"\nTesting {test_code_name} ({test_code_type}) across noise levels with all decoders...\n")

noise_levels = [0.001, 0.005, 0.01, 0.02]

# Header
dec_names = list(decoder_classes.keys())
header = f"{'p':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:12]:>12}"
header += " | No-decode"
print(header)
print("-" * len(header))

for p in noise_levels:
    row = f"{p:<10.4f}"
    ler_no_decode = None
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=test_code,
            code_type=test_code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=2000,
            rounds=3
        )
        
        if ler_no_decode is None and result.get('ler_no_decode'):
            ler_no_decode = result['ler_no_decode']
        
        ler = result.get('ler')
        ler_str = f"{ler:.6f}" if ler else 'FAIL'
        row += f" | {ler_str:>12}"
    
    nd_str = f"{ler_no_decode:.6f}" if ler_no_decode else 'N/A'
    row += f" | {nd_str}"
    print(row)

print("-" * len(header))

NOISE LEVEL SCALING TEST

Testing RotatedSurface_[[9,1,3]] (CSS) across noise levels with all decoders...

p          |   PyMatching | FusionBlosso | BeliefMatchi |        BPOSD |    Tesseract |    UnionFind |          MLE |   Hypergraph |   Chromobius | No-decode
-------------------------------------------------------------------------------------------------------------------------------------------------------------
0.0010     |         FAIL |     0.000500 |     0.000500 |     0.000500 |         FAIL |         FAIL |         FAIL |     0.008500 |         FAIL | 0.012500
0.0010     |         FAIL |     0.000500 |     0.000500 |     0.000500 |         FAIL |         FAIL |         FAIL |     0.008500 |         FAIL | 0.012500
0.0050     |     0.004500 |     0.005000 |     0.004500 |     0.004500 |     0.003500 |     0.002000 |     0.004000 |     0.050500 |         FAIL | 0.051000
0.0050     |     0.004500 |     0.005000 |     0.004500 |     0.004500 |     0.003500 |     0.002000 |    

In [6]:
# Cell 8: FINAL SUMMARY - ALL CODE TYPES

print("="*100)
print("FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES")
print("="*100)

# Count results by status
passed = 0
with_warnings = 0
skipped = 0
failed = 0
total_tests = 0

# Track by code type - include Color codes
results_by_type = {'CSS': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Color': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Non-CSS': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Subsystem': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'Floquet': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0},
                   'QLDPC': {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0}}

failed_tests = []

for code_name, code_results in full_results.items():
    code_type = code_results.get('type', 'Unknown')
    if code_type not in results_by_type:
        results_by_type[code_type] = {'pass': 0, 'warn': 0, 'skip': 0, 'fail': 0}
    
    for dec_name in decoder_classes.keys():
        res = code_results.get(dec_name, {})
        if isinstance(res, dict):
            total_tests += 1
            status = res.get('status')
            if status == 'OK':
                if res.get('warnings'):
                    with_warnings += 1
                    results_by_type[code_type]['warn'] += 1
                else:
                    passed += 1
                    results_by_type[code_type]['pass'] += 1
            elif status == 'WARN':
                # Expected limitation (decoder incompatibility, non-graphlike DEM, etc.)
                with_warnings += 1
                results_by_type[code_type]['warn'] += 1
            elif status == 'SKIP':
                # Expected: decoder not designed for this code type (e.g., Chromobius on non-color)
                skipped += 1
                results_by_type[code_type]['skip'] += 1
            else:
                failed += 1
                results_by_type[code_type]['fail'] += 1
                failed_tests.append((code_name, dec_name, res.get('error', 'unknown')))

print(f"\n📊 OVERALL RESULTS:")
print(f"   Total decoder × code tests: {total_tests}")
print(f"   ✓ Passed:   {passed} ({100*passed/total_tests:.1f}%)")
print(f"   ⚠ Warnings: {with_warnings} ({100*with_warnings/total_tests:.1f}%)")
print(f"   - Skipped:  {skipped} ({100*skipped/total_tests:.1f}%) [expected - decoder incompatible]")
print(f"   ✗ Failed:   {failed} ({100*failed/total_tests:.1f}%)")

print(f"\n📊 RESULTS BY CODE TYPE:")
for ctype, counts in results_by_type.items():
    total = counts['pass'] + counts['warn'] + counts['skip'] + counts['fail']
    if total > 0:
        chromobius_note = " (Chromobius works!)" if ctype == 'Color' else ""
        print(f"   {ctype:>10}: {counts['pass']} pass, {counts['warn']} warn, {counts['skip']} skip, {counts['fail']} fail (total {total}){chromobius_note}")

print(f"\n🔧 DECODERS TESTED: {len(decoder_classes)}")
for dec_name in decoder_classes.keys():
    note = " (works on Color codes now!)" if dec_name == "Chromobius" else ""
    print(f"   - {dec_name}{note}")

print(f"\n📦 CODES TESTED BY TYPE:")
for code_type, codes_dict in [('CSS', css_codes), ('Color (Chromobius)', color_codes),
                               ('Non-CSS', non_css_codes), ('Subsystem', subsystem_codes), 
                               ('Floquet', floquet_codes), ('QLDPC', qldpc_codes)]:
    if codes_dict:
        print(f"\n   {code_type} ({len(codes_dict)} codes):")
        for name, code in codes_dict.items():
            d = code.metadata.get('distance', '?')
            print(f"     - {name}: [[{code.n},{code.k},{d}]]")

print("\n" + "="*100)
if failed == 0:
    print("✓ ALL TESTS PASSED (no unexpected failures)")
    if skipped > 0:
        print(f"  ({skipped} tests skipped - Chromobius skipped on non-color codes)")
else:
    print(f"⚠ {failed} TESTS FAILED (unexpected):")
    for code_name, dec_name, error in failed_tests[:20]:  # Show first 20
        print(f"   - {code_name} × {dec_name}: {error}")
    if len(failed_tests) > 20:
        print(f"   ... and {len(failed_tests) - 20} more")
print("="*100)

FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES

📊 OVERALL RESULTS:
   Total decoder × code tests: 585
   ✓ Passed:   328 (56.1%)
   ⚠ Warnings: 81 (13.8%)
   - Skipped:  52 (8.9%) [expected - decoder incompatible]
   ✗ Failed:   124 (21.2%)

📊 RESULTS BY CODE TYPE:
          CSS: 243 pass, 34 warn, 38 skip, 117 fail (total 432)
        Color: 16 pass, 16 warn, 1 skip, 3 fail (total 36) (Chromobius works!)
      Non-CSS: 24 pass, 16 warn, 5 skip, 0 fail (total 45)
    Subsystem: 16 pass, 15 warn, 4 skip, 1 fail (total 36)
      Floquet: 8 pass, 0 warn, 1 skip, 0 fail (total 9)
        QLDPC: 21 pass, 0 warn, 3 skip, 3 fail (total 27)

🔧 DECODERS TESTED: 9
   - PyMatching
   - FusionBlossom
   - BeliefMatching
   - BPOSD
   - Tesseract
   - UnionFind
   - MLE
   - Hypergraph
   - Chromobius (works on Color codes now!)

📦 CODES TESTED BY TYPE:

   CSS (48 codes):
     - FourQubit422_[[4,2,2]]: [[4,2,2]]
     - C6: [[6,2,2]]
     - Steane_713: [[7,1,3]]
     - Shor_91: [[9,1,3]]
     - 

In [9]:
# Cell 9: CHROMOBIUS DIRECT TEST ON COLOR CODES
# This cell tests Chromobius directly on color codes using ColorCodeMemoryExperiment
# which generates 4D detector coordinates with proper color annotations

import sys
# Force reimport to pick up latest code changes
modules_to_remove = [k for k in sys.modules.keys() if 'qectostim' in k]
for mod in modules_to_remove:
    del sys.modules[mod]

from qectostim.codes.color.triangular_colour import TriangularColourCode
from qectostim.codes.color.hexagonal_colour import HexagonalColourCode
from qectostim.codes.color.colour_code import ColourCode488
from qectostim.experiments.memory import ColorCodeMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
import chromobius
import numpy as np

print("="*60)
print("CHROMOBIUS DIRECT TEST ON COLOR CODES")
print("="*60)
print("Uses ColorCodeMemoryExperiment for 4D detector coordinates")
print()

codes = [
    ('TriangularColour_d3', TriangularColourCode(distance=3)),
    ('HexagonalColour_d2', HexagonalColourCode(distance=2)),
    ('HexagonalColour_d3', HexagonalColourCode(distance=3)),
    ('Colour488_d3', ColourCode488(distance=3)),
]

shots = 1000
results = []

for name, code in codes:
    print(f"\n{'-'*50}")
    print(f"Testing: {name} (n={code.n}, k={code.k})")
    meta = code.metadata
    print(f"  stab_colors: {meta.get('stab_colors')}")
    print(f"  is_chromobius_compatible: {meta.get('is_chromobius_compatible')}")
    
    if not meta.get('is_chromobius_compatible', False):
        print(f"  ⚠️  SKIPPED - not 3-colorable (stabilizers form K4 complete graph)")
        results.append((name, 'SKIP', 'Not 3-colorable'))
        continue
    
    try:
        noise = CircuitDepolarizingNoise(p1=0.01, p2=0.01)
        exp = ColorCodeMemoryExperiment(code=code, rounds=3, noise_model=noise)
        circuit = noise.apply(exp.to_stim())
        
        # Sample from circuit directly
        sampler = circuit.compile_detector_sampler()
        dets, obs = sampler.sample(shots, separate_observables=True)
        
        # Compile Chromobius decoder
        dem = circuit.detector_error_model(decompose_errors=True)
        decoder = chromobius.compile_decoder_for_dem(dem)
        
        # Decode
        dets_packed = np.packbits(dets.astype(np.uint8), axis=1, bitorder='little')
        predictions = decoder.predict_obs_flips_from_dets_bit_packed(dets_packed)
        
        # Calculate error rate
        obs = obs.astype(np.uint8)
        errors = np.any(predictions != obs, axis=1).sum()
        ler = errors / shots
        print(f"  ✓ Chromobius decode: SUCCESS - LER = {ler:.4f} ({errors}/{shots} errors)")
        results.append((name, 'PASS', f'{ler:.4f}'))
        
    except Exception as e:
        print(f"  ✗ ERROR: {type(e).__name__}: {e}")
        results.append((name, 'FAIL', str(e)[:50]))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for name, status, detail in results:
    emoji = {'PASS': '✓', 'SKIP': '⚠️', 'FAIL': '✗'}[status]
    print(f"  {emoji} {name}: {status} - {detail}")

CHROMOBIUS DIRECT TEST ON COLOR CODES
Uses ColorCodeMemoryExperiment for 4D detector coordinates


--------------------------------------------------
Testing: TriangularColour_d3 (n=7, k=1)
  stab_colors: [0, 1, 2]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0190 (19/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d2 (n=8, k=2)
  stab_colors: [0, 0, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.0000 (0/1000 errors)

--------------------------------------------------
Testing: HexagonalColour_d3 (n=17, k=7)
  stab_colors: [0, 0, 0, 1, 1]
  is_chromobius_compatible: True
  ✓ Chromobius decode: SUCCESS - LER = 0.1570 (157/1000 errors)

--------------------------------------------------
Testing: Colour488_d3 (n=9, k=1)
  stab_colors: [0, 1, 2, 0]
  is_chromobius_compatible: False
  ⚠️  SKIPPED - not 3-colorable (stabilizers form K4 complete graph)

SUMMARY
  ✓ TriangularColour_d3: PASS - 0

In [10]:
# Debug: Try to instantiate the fracton codes directly
import sys
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

print("="*70)
print("TRYING TO INSTANTIATE FRACTON CODES DIRECTLY")
print("="*70)

# Import topological module
from qectostim.codes import topological

# Try each fracton code
fracton_classes = ['HaahCode', 'XCubeCode', 'ChamonCode', 'CheckerboardCode', 
                    'FibonacciFractalCode', 'SierpinskiPrismCode']

for class_name in fracton_classes:
    cls = getattr(topological, class_name, None)
    if cls is None:
        print(f"❌ {class_name}: NOT FOUND")
        continue
    
    # Try different parameter combinations
    params_to_try = [
        {'L': 3},
        {'L': 4},
        {'L': 3, 'T': 2},
        {},
    ]
    
    for params in params_to_try:
        try:
            code = cls(**params) if params else cls()
            print(f"✓ {class_name}({params}): n={code.n}, k={code.k}")
            break
        except Exception as e:
            print(f"  {class_name}({params}): {type(e).__name__}: {str(e)[:50]}")

# Also try HoneycombCode
print("\n" + "="*70)
print("TRYING FLOQUET CODES")
print("="*70)

from qectostim.codes import floquet

for class_name in ['HoneycombCode', 'ISGFloquetCode']:
    cls = getattr(floquet, class_name, None)
    if cls is None:
        print(f"❌ {class_name}: NOT FOUND")
        continue
    
    params_to_try = [
        {'Lx': 2, 'Ly': 3},
        {'distance': 3},
        {'L': 3},
        {},
    ]
    
    for params in params_to_try:
        try:
            code = cls(**params) if params else cls()
            print(f"✓ {class_name}({params}): n={code.n}, k={code.k}")
            break
        except Exception as e:
            print(f"  {class_name}({params}): {type(e).__name__}: {str(e)[:60]}")

TRYING TO INSTANTIATE FRACTON CODES DIRECTLY
✓ HaahCode({'L': 3}): n=54, k=2
✓ XCubeCode({'L': 3}): n=81, k=-17
✓ ChamonCode({'L': 3}): n=54, k=2
  CheckerboardCode({'L': 3}): ValueError: L must be even and >= 2
✓ CheckerboardCode({'L': 4}): n=64, k=38
  FibonacciFractalCode({'L': 3}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
  FibonacciFractalCode({'L': 4}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
  FibonacciFractalCode({'L': 3, 'T': 2}): TypeError: FibonacciFractalCode.__init__() got an unexpected 
✓ FibonacciFractalCode({}): n=23, k=1
  SierpinskiPrismCode({'L': 3}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
  SierpinskiPrismCode({'L': 4}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
  SierpinskiPrismCode({'L': 3, 'T': 2}): TypeError: SierpinskiPrismCode.__init__() got an unexpected k
✓ SierpinskiPrismCode({}): n=80, k=1

TRYING FLOQUET CODES
  HoneycombCode({'Lx': 2, 'Ly': 3}): TypeError: HoneycombCode

In [7]:
# Cell: Verify fixes for k<=0 and d=? codes
import sys
# Force reimport
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

from qectostim.codes import discover_all_codes

print("="*70)
print("VERIFYING k AND d FIXES")
print("="*70)

codes = discover_all_codes(max_qubits=100, timeout_per_code=3.0)

print(f"\nTotal codes discovered: {len(codes)}")

# Check for problematic codes
k_zero_codes = []
k_negative_codes = []
d_unknown_codes = []
all_good_codes = []

for name, code in codes.items():
    k = code.k
    d = code.metadata.get('distance', None)
    
    if k < 0:
        k_negative_codes.append((name, k, d))
    elif k == 0:
        k_zero_codes.append((name, k, d))
    elif d is None or d == '?':
        d_unknown_codes.append((name, k, d))
    else:
        all_good_codes.append((name, k, d))

print(f"\n✅ Codes with k>0 and known d: {len(all_good_codes)}")
print(f"⚠️  Codes with k=0: {len(k_zero_codes)}")
print(f"❌ Codes with k<0: {len(k_negative_codes)}")
print(f"⚠️  Codes with unknown d: {len(d_unknown_codes)}")

if k_negative_codes:
    print(f"\n❌ NEGATIVE K CODES (still broken):")
    for name, k, d in k_negative_codes:
        print(f"   {name}: k={k}, d={d}")

if k_zero_codes:
    print(f"\n⚠️  ZERO K CODES:")
    for name, k, d in k_zero_codes:
        print(f"   {name}: k={k}, d={d}")

if d_unknown_codes:
    print(f"\n⚠️  UNKNOWN DISTANCE CODES:")
    for name, k, d in d_unknown_codes[:10]:
        print(f"   {name}: k={k}, d={d}")
    if len(d_unknown_codes) > 10:
        print(f"   ... and {len(d_unknown_codes) - 10} more")

print("\n" + "="*70)
if len(k_negative_codes) == 0:
    print("✅ NO NEGATIVE K CODES - XCubeCode fix verified!")
else:
    print("❌ Still have negative k codes - needs investigation")

if len(k_zero_codes) <= 2:  # Some codes legitimately have k=0
    print("✅ ZERO K CODES MINIMIZED")
else:
    print(f"⚠️  Still have {len(k_zero_codes)} k=0 codes")

print("="*70)

VERIFYING k AND d FIXES

Total codes discovered: 62

✅ Codes with k>0 and known d: 48
⚠️  Codes with k=0: 0
❌ Codes with k<0: 0
⚠️  Codes with unknown d: 14

⚠️  UNKNOWN DISTANCE CODES:
   ProjectivePlaneSurface_[[13,1,None]]: k=1, d=None
   HGPHamming7_[[58,16,None]]: k=16, d=None
   CampbellDoubleHGP_3_[[13,1,None]]: k=1, d=None
   CampbellDoubleHGP_5_[[41,1,None]]: k=1, d=None
   GaloisQuditSurface_[[13,1,?]]: k=1, d=None
   GaloisQuditSurface_[[25,1,?]]: k=1, d=None
   GaloisQuditColor_[[25,1,?]]: k=1, d=None
   ModularQuditColorCode_L3_d3: k=1, d=None
   IntegerHomologyBosonic_[[25,1,?]]: k=1, d=None
   IntegerHomologyBosonic_[[41,1,?]]: k=1, d=None
   ... and 4 more

✅ NO NEGATIVE K CODES - XCubeCode fix verified!
✅ ZERO K CODES MINIMIZED
